This is a script to compare the scdesigner AIC/BIC result to the scDesign3 AIC/BIC result.

In [1]:
import anndata
import os
import requests

save_path = "data/example_sce.h5ad"
if not os.path.exists(save_path):
    response = requests.get("https://go.wisc.edu/69435h")
    with open(save_path, "wb") as f:
        f.write(response.content)

example_sce = anndata.read_h5ad(save_path)
example_sce

AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'
    var: 'highly_variable_genes'
    uns: 'X_name', 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca'
    obsm: 'PCA', 'UMAP', 'X_pca', 'X_umap'
    layers: 'counts', 'cpm', 'logcounts', 'spliced', 'unspliced'
    obsp: 'connectivities', 'distances'

# scDesign3 impelentation

I will first calculate the AIC/BIC in scDesign3 under similar model setup.

In [2]:
%load_ext rpy2.ipython

Error importing in API mode: ImportError("dlopen(/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/_rinterface_cffi_api.abi3.so, 0x0002): Library not loaded: /Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib\n  Referenced from: <3D361A20-57F1-3C9B-9D2B-6A0F70CF5F35> /Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/_rinterface_cffi_api.abi3.so\n  Reason: tried: '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file)")
Trying to import in ABI mode.
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "PWD" redefined by R and overriding existing variable. Current: "/", R: "/Users/pyl/Desktop/UW Madis

In [3]:
%%R
library(scDesign3)
library(ggplot2)
library(zellkonverter)
theme_set(theme_bw())

example_sce <- readH5AD("data/example_sce.h5ad")


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    ℹ Using stored X_name value 'X'


Registered S3 method overwritten by 'scDesign3':
  method         from  
  predict.gamlss gamlss
Registered S3 method overwritten by 'zellkonverter':
  method                                             from      
  py_to_r.pandas.core.arrays.categorical.Categorical reticulate


In [4]:
%%R
example_simu <- scdesign3(
  sce = example_sce,
  assay_use = "counts",
  celltype = "cell_type",
  pseudotime = "pseudotime",
  spatial = NULL,
  other_covariates = NULL,
  mu_formula = "s(pseudotime, k = 6, bs = 'cr')",
  sigma_formula = "1",
  family_use = "nb",
  n_cores = 2,
  usebam = FALSE,
  corr_formula = "1",
  copula = "gaussian",
  DT = TRUE,
  pseudo_obs = FALSE,
  return_model = FALSE,
  nonzerovar = FALSE
) 

Input Data Construction Start
Input Data Construction End
Start Marginal Fitting
Marginal Fitting End
Start Copula Fitting
Convert Residuals to Multivariate Gaussian
Converting End
Copula group 1 starts
Copula Fitting End
Start Parameter Extraction
Parameter
Extraction End
Start Generate New Data
Use Copula to sample a multivariate quantile matrix
Sample Copula group 1 starts
New Data Generating End


And below are the marginal and gaussian copula AIC/BIC calculated directly from scDesign3.

In [5]:
%%R
print(paste0("marginal AIC: ", example_simu$model_aic[1]))
print(paste0("marginal BIC: ", example_simu$model_bic[1]))
print(paste0("copula AIC: ", example_simu$model_aic[2]))
print(paste0("copula BIC: ", example_simu$model_bic[2]))

[1] "marginal AIC: 778898.707033089"
[1] "marginal BIC: 782773.03646551"
[1] "copula AIC: -8982.81517708313"
[1] "copula BIC: 18952.425212949"


# scdesigner AIC/BIC example

Now we are going to extract AIC/BIC from our scdesigner models.

In [6]:
from scdesigner.simulators import NegBinCopulaSimulator

/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `annda

In [7]:
formula = "~ bs(pseudotime, degree=5)"
nbcopula = NegBinCopulaSimulator()
nbcopula.fit(example_sce, formula)

/Users/pyl/Desktop/UW Madison/Graduate/2025 Summer/sc project/main/scDesigner/scdesigner/src/scdesigner/data/group.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.list[idx]
Epoch 1/40:   0%|          | 0/5 [00:00<?, ?it/s]/Users/pyl/Desktop/UW Madison/Graduate/2025 Summer/sc project/main/scDesigner/scdesigner/src/scdesigner/data/group.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.list[idx]


Marginal and copula AIC and BIC are included as instances within each simulator class.

In [8]:
print("Marginal AIC:", nbcopula.marginal_aic)
print("Marginal AIC:", nbcopula.marginal_bic)
print("Copula AIC:", nbcopula.copula_aic)
print("Copula BIC:", nbcopula.copula_bic)

Marginal AIC: 793849.9375
Marginal AIC: 797800.375
Copula AIC: -9314.957550434978
Copula BIC: 18620.282839597166


Which are also the same as what we got from the previous version.